# Importing Necessary Libraries

In [2]:
! pip install tensorflow==1.15

  Using cached tensorflow-1.15.0-cp37-cp37m-manylinux2010_x86_64.whl (412.3 MB)
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 1.15.4
    Uninstalling tensorflow-1.15.4:
      Successfully uninstalled tensorflow-1.15.4


In [4]:
!pip install keras==2.3.1

  Using cached Keras-2.3.1-py2.py3-none-any.whl (377 kB)
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0


In [81]:
import pandas as pd
import numpy as np
np.random.seed(1337)
from tensorflow import keras 
from keras import Sequential
from keras.utils import Sequence
from tensorflow.keras.utils import Sequence
from keras.layers import LSTM, Dense, Masking
import numpy as np
import keras
from keras.utils import np_utils
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Embedding, Dense, Input, concatenate, Layer, Lambda, Dropout, Activation
import datetime
from datetime import datetime
from keras.callbacks import ModelCheckpoint, EarlyStopping, Callback, TensorBoard
import tensorflow as tf
import tensorflow_hub as hub

# Loading The Data

In [108]:
train_raw = pd.read_csv('train_6128.csv')
test_raw = pd.read_csv('test_1532.csv')
train_raw = pd.concat([train_raw, test_raw])
train_raw.head()

,Unnamed: 0,Text,MLclass
0,5860,1\r\ni SUPPLIES & SOLUTIONS FOR INDUSTRY\r\nww...,single
1,6007,"\r\nC. BEAN TRANSPORT, INC.\r\n(VAN DIVISION)\...",single
2,1192,"support@almo.com Tuesday, June 10, 2014 1:46 P...",single
3,6214,"1\r\nMuller Martini 456 Wheeler Rd Hauppauge, ...",single
4,4231,"INVOICE\r\nAndrews\r\nAndrews Paperboard, Inc....",single


In [109]:
train_raw.sort_values("Text", inplace = True)
 
# dropping ALL duplicate values
train_raw.drop_duplicates(subset ="Text",keep = False, inplace = True)

In [110]:
train_raw.shape

(4702, 3)

# Preprocessing Data

Select non null:

In [111]:
train_raw = train_raw[train_raw.Text.notnull()]
train_raw.shape

(4701, 3)

In [112]:
train_raw.Text.apply(lambda x: len(x.split())).plot(kind='hist')

<AxesSubplot:ylabel='Frequency'>

In [113]:
train_raw['len_txt'] =train_raw.Text.apply(lambda x: len(x.split()))
train_raw.describe()

,Unnamed: 0,len_txt
count,4701.000000,4701.000000
mean,4071.061051,190.693469
std,2129.833330,99.931765
min,0.000000,1.000000
25%,2559.000000,121.000000
50%,4318.000000,174.000000
75%,5725.000000,244.000000
max,7656.000000,1899.000000


In [114]:
train_raw.shape

(4701, 4)

Select only the row with number of words greater than 250:

In [115]:
#train_raw = train_raw[train_raw.len_txt >249]
#train_raw.shape

Group similar products

In [116]:
for l in np.unique(train_raw['MLclass']):
  print(l)

first
intermediate
last
single


In [117]:
train_raw['MLclass'].value_counts().sort_values(ascending=False).plot(kind='bar')

<AxesSubplot:ylabel='Frequency'>

In [118]:
train_raw=train_raw.rename(columns = {'Text':'text', 'MLclass':'label'})
train_raw.head()

,Unnamed: 0,text,label,len_txt
287,1103,\t\t\t\t\t\t\t\tPRO NUMBER\t1\r\nfRAILER NUMBE...,single,164
3863,1058,\t\t\t\t\t\r\nHALOPHANE CORP - S DOBSON 214 OA...,single,325
2373,7630,\t\t\tInvoice #\tInv.Date\tPage\r\n< 0 (/)\tN ...,first,257
5432,7238,\t\tINVOICE * RENEWAL STORAGE *\t\t\t\t\tPAGE ...,first,321
103,4834,\t3M Invoice\t\t\t\t\t\t\r\nPAGE\t1 OF\t1\tPUR...,single,223


In [119]:
from sklearn.preprocessing import LabelEncoder

LE = LabelEncoder()
train_raw['label'] = LE.fit_transform(train_raw['label'])
train_raw.head()

,Unnamed: 0,text,label,len_txt
287,1103,\t\t\t\t\t\t\t\tPRO NUMBER\t1\r\nfRAILER NUMBE...,3,164
3863,1058,\t\t\t\t\t\r\nHALOPHANE CORP - S DOBSON 214 OA...,3,325
2373,7630,\t\t\tInvoice #\tInv.Date\tPage\r\n< 0 (/)\tN ...,0,257
5432,7238,\t\tINVOICE * RENEWAL STORAGE *\t\t\t\t\tPAGE ...,0,321
103,4834,\t3M Invoice\t\t\t\t\t\t\r\nPAGE\t1 OF\t1\tPUR...,3,223


In [120]:
len(np.unique(train_raw['label']))

4

In [121]:
train_raw = train_raw[["text", "label"]]

In [122]:
train_raw.head()

,text,label
287,\t\t\t\t\t\t\t\tPRO NUMBER\t1\r\nfRAILER NUMBE...,3
3863,\t\t\t\t\t\r\nHALOPHANE CORP - S DOBSON 214 OA...,3
2373,\t\t\tInvoice #\tInv.Date\tPage\r\n< 0 (/)\tN ...,0
5432,\t\tINVOICE * RENEWAL STORAGE *\t\t\t\t\tPAGE ...,0
103,\t3M Invoice\t\t\t\t\t\t\r\nPAGE\t1 OF\t1\tPUR...,3


In [123]:
train = train_raw.copy()

In [126]:

 train

,text,label
287,\t\t\t\t\t\t\t\tPRO NUMBER\t1\r\nfRAILER NUMBE...,3
3863,\t\t\t\t\t\r\nHALOPHANE CORP - S DOBSON 214 OA...,3
2373,\t\t\tInvoice #\tInv.Date\tPage\r\n< 0 (/)\tN ...,0
5432,\t\tINVOICE * RENEWAL STORAGE *\t\t\t\t\tPAGE ...,0
103,\t3M Invoice\t\t\t\t\t\t\r\nPAGE\t1 OF\t1\tPUR...,3
...,...,...
239,■CUSTOMERS AGREE TO THE GENERAL STONEPEAK CERA...,3
5904,■n DCKERLUlfD 1 EfD U S'TRIE S comp my\r\n^Qce...,3
1488,■worcH\r\nMALIN\r\n. RAVMOND...\r\nN.J. Malin ...,3
1205,■■■\r\n■ IlM\r\n■Mtu\r\nVALASSIS COMMUNICATION...,3


In [131]:
train = train.reset_index(drop=True)

In [132]:
train = train.reindex(np.random.permutation(train.index))
train.head()

,text,label
1572,GJ Westaff\r\nPROVIDING ESSENTIAL PEOPLE\r\nRe...,3
694,AUERITT EXPRESS\t956 7969313 P.07/20\r\nAVERIT...,3
1390,"Echo Lane, Inc.\r\n665 Third Street, Suite 519...",3
4334,gettyimages®\r\nLine No. Description\tOrdered ...,1
1676,I>i V V-/I<->L_\r\nFor Remittance Only:\tL\r\n...,3


Clean the text columns

In [133]:
import re
def clean_txt(text):
  text = re.sub("'", "",text)
  text=re.sub("(\\W)+"," ",text)    
  return text

In [134]:
train['text']  = train.text.apply(clean_txt)
train.head()

,text,label
1572,GJ Westaff PROVIDING ESSENTIAL PEOPLE Remit To...,3
694,AUERITT EXPRESS 956 7969313 P 07 20 AVERITT EX...,3
1390,Echo Lane Inc 665 Third Street Suite 519 San F...,3
4334,gettyimages Line No Description Ordered Unit P...,1
1676,I i V V I L_ For Remittance Only L CONVEYORS I...,3


In [135]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(train, test_size=0.2, random_state=35)
train.head()

,text,label
164,VVeritiv feritrv Operating Company elnvoice F...,3
3708,SKS Transport Inc 573 W Babcock Avenue Elmhurs...,3
2249,Invoice 400 Ahnaip Street Suite 200 Menasha Wl...,3
1652,Hiease nemn i o NW5525 PO BOX 1450 MINNEAPOLIS...,3
3905,THE SHERWIN WILLIAMS CO 200 SYCAMORE ST STE 11...,3


In [136]:
train.reset_index(drop=True, inplace=True)
train.head(2)

,text,label
0,VVeritiv feritrv Operating Company elnvoice F...,3
1,SKS Transport Inc 573 W Babcock Avenue Elmhurs...,3


In [137]:
val.reset_index(drop=True, inplace=True)
val.head(2)

,text,label
0,Invoice wrh global Americas 24 Worlds Fair Dri...,3
1,REMIT TO A TA equipment MUM COMPANY SALES INVO...,3


In [138]:
val.shape, train.shape

((941, 2), (3760, 2))

In [140]:
#Importing BERT modules
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

# Setting The Output Directory for BERT

In [141]:
# Set the output directory for saving model file
OUTPUT_DIR = './bert_news_category'

#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = True #@param {type:"boolean"}

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    pass

tf.gfile.MakeDirs(OUTPUT_DIR)
#tf.io.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

***** Model output directory: ./bert_news_category *****


In [142]:
print("Training Set Shape :", train.shape)
print("Validation Set Shape :", val.shape)
# print("Test Set Shape :", test.shape)

Training Set Shape : (3760, 2)
Validation Set Shape : (941, 2)


In [143]:
DATA_COLUMN = 'text'
LABEL_COLUMN = 'label'
# The list containing all the classes (train['SECTION'].unique())
label_list = [x for x in np.unique(train.label)]
label_list

[0, 1, 2, 3]

# Splitting the Data into smaller chunks

In [144]:
def get_split(text1):
  l_total = []
  l_parcial = []
  if len(text1.split())//150 >0:
    n = len(text1.split())//150
  else: 
    n = 1
  for w in range(n):
    if w == 0:
      l_parcial = text1.split()[:200]
      l_total.append(" ".join(l_parcial))
    else:
      l_parcial = text1.split()[w*150:w*150 + 200]
      l_total.append(" ".join(l_parcial))
  return l_total

In [145]:
train['text_split'] = train[DATA_COLUMN].apply(get_split)
train.head()

,text,label,text_split
0,VVeritiv feritrv Operating Company elnvoice F...,3,[VVeritiv feritrv Operating Company elnvoice F...
1,SKS Transport Inc 573 W Babcock Avenue Elmhurs...,3,[SKS Transport Inc 573 W Babcock Avenue Elmhur...
2,Invoice 400 Ahnaip Street Suite 200 Menasha Wl...,3,[Invoice 400 Ahnaip Street Suite 200 Menasha W...
3,Hiease nemn i o NW5525 PO BOX 1450 MINNEAPOLIS...,3,[Hiease nemn i o NW5525 PO BOX 1450 MINNEAPOLI...
4,THE SHERWIN WILLIAMS CO 200 SYCAMORE ST STE 11...,3,[THE SHERWIN WILLIAMS CO 200 SYCAMORE ST STE 1...


In [146]:
val['text_split'] = val[DATA_COLUMN].apply(get_split)
val.head(2)

,text,label,text_split
0,Invoice wrh global Americas 24 Worlds Fair Dri...,3,[Invoice wrh global Americas 24 Worlds Fair Dr...
1,REMIT TO A TA equipment MUM COMPANY SALES INVO...,3,[REMIT TO A TA equipment MUM COMPANY SALES INV...


In [147]:
train_l = []
label_l = []
index_l =[]
for idx,row in train.iterrows():
  for l in row['text_split']:
    train_l.append(l)
    label_l.append(row['label'])
    index_l.append(idx)
len(train_l), len(label_l), len(index_l)

(4623, 4623, 4623)

In [148]:
val_l = []
val_label_l = []
val_index_l = []
for idx,row in val.iterrows():
  for l in row['text_split']:
    val_l.append(l)
    val_label_l.append(row['label'])
    val_index_l.append(idx)
len(val_l), len(val_label_l), len(val_index_l)

(1154, 1154, 1154)

The final dataset for training:

In [149]:
train_df = pd.DataFrame({DATA_COLUMN:train_l, LABEL_COLUMN:label_l})
train_df.head()

,text,label
0,VVeritiv feritrv Operating Company elnvoice FS...,3
1,Description References Weight Qty Ordered Qty ...,3
2,SKS Transport Inc 573 W Babcock Avenue Elmhurs...,3
3,Invoice 400 Ahnaip Street Suite 200 Menasha Wl...,3
4,Hiease nemn i o NW5525 PO BOX 1450 MINNEAPOLIS...,3


In [150]:
val_df = pd.DataFrame({DATA_COLUMN:val_l, LABEL_COLUMN:val_label_l})
val_df.head()

,text,label
0,Invoice wrh global Americas 24 Worlds Fair Dri...,3
1,REMIT TO A TA equipment MUM COMPANY SALES INVO...,3
2,Breison International Inc 4540 W 160 St Clevel...,3
3,BakeryCorp Miami FL 15625 NW 15Th Avenue Miami...,3
4,NEXVUE IXPgCT M08f ____ 65 Broad Street Stamfo...,2


# BERT: Data Preprocessing 

Process the data for BERT

In [151]:
train_InputExamples = train_df.apply(lambda x: bert.run_classifier.InputExample(guid=None,
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

val_InputExamples = val_df.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [152]:
train_InputExamples

0       <bert.run_classifier.InputExample object at 0x...
1       <bert.run_classifier.InputExample object at 0x...
2       <bert.run_classifier.InputExample object at 0x...
3       <bert.run_classifier.InputExample object at 0x...
4       <bert.run_classifier.InputExample object at 0x...
                              ...                        
4618    <bert.run_classifier.InputExample object at 0x...
4619    <bert.run_classifier.InputExample object at 0x...
4620    <bert.run_classifier.InputExample object at 0x...
4621    <bert.run_classifier.InputExample object at 0x...
4622    <bert.run_classifier.InputExample object at 0x...
Length: 4623, dtype: object

In [153]:
print("Row 0 - guid of training set : ", train_InputExamples.iloc[0].guid)
print("\n__________\nRow 0 - text_a of training set : ", train_InputExamples.iloc[0].text_a)
print("\n__________\nRow 0 - text_b of training set : ", train_InputExamples.iloc[0].text_b)
print("\n__________\nRow 0 - label of training set : ", train_InputExamples.iloc[0].label)

Row 0 - guid of training set :  None

__________
Row 0 - text_a of training set :  VVeritiv feritrv Operating Company elnvoice FSC SGSNA COC 003630 SFI SGS SFI COC 0037 PEFC SGS PEFC COC 0208 VERITIV OPERATING COMPANY 6120 South Gilmore Road Fairfield OH 45014 VALASSIS PO BOX 159 VENDOR FINANCIAL SERVICES WINDSOR CT 06095 0159 Invoice No Invoice Date Account No 579 75477630 05 02 2019 313050 Due Date Total Amount Due Enter Amount Paid 06 01 2019 338 86 VERITIV OPERATING COMPANY 7472 COLLECTION CENTER DRIVE CHICAGO IL 60693 Detach and return this portion with remittance Retain bottom portion for your records S H I P T 0 VALASSIS INSERTS 35955 SCHOOLCRAFT LIVONIA MI 48150 1215 S 0 L D T 0 Invoice No Invoice Date Customer P O Order Date VALASSIS PO BOX 159 VENDOR FINANCIAL SERVICES WINDSOR CT 06095 0159 Page 1 1 Type of Sale WAREHOUSE Terms 579 75477630 05 02 2019 1062167 05 01 2019 NET 30 DAYS Product No Description References Weight Qty Ordered Qty Shipped Unit Meas Unit Price Unit Exte

# BERT: Loading the pre-trained model

In [154]:

# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

In [155]:
len(tokenizer.vocab.keys())

30522

In [156]:
#Here is what the tokenised sample of the first training set observation looks like
print(tokenizer.tokenize(train_InputExamples.iloc[0].text_a))

['v', '##ver', '##iti', '##v', 'fe', '##rit', '##r', '##v', 'operating', 'company', 'el', '##n', '##vo', '##ice', 'f', '##sc', 'sg', '##s', '##na', 'co', '##c', '00', '##36', '##30', 'sf', '##i', 'sg', '##s', 'sf', '##i', 'co', '##c', '00', '##37', 'pe', '##fc', 'sg', '##s', 'pe', '##fc', 'co', '##c', '02', '##0', '##8', 've', '##rit', '##iv', 'operating', 'company', '61', '##20', 'south', 'gilmore', 'road', 'fairfield', 'oh', '450', '##14', 'val', '##ass', '##is', 'po', 'box', '159', 'vendor', 'financial', 'services', 'windsor', 'ct', '06', '##0', '##9', '##5', '01', '##59', 'in', '##vo', '##ice', 'no', 'in', '##vo', '##ice', 'date', 'account', 'no', '57', '##9', '75', '##47', '##7', '##6', '##30', '05', '02', '2019', '313', '##0', '##50', 'due', 'date', 'total', 'amount', 'due', 'enter', 'amount', 'paid', '06', '01', '2019', '338', '86', 've', '##rit', '##iv', 'operating', 'company', '747', '##2', 'collection', 'center', 'drive', 'chicago', 'il', '60', '##6', '##9', '##3', 'det', '##

In [157]:
#!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [158]:
MAX_SEQ_LENGTH = 200

In [159]:
# Convert our train and validation features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

val_features = bert.run_classifier.convert_examples_to_features(val_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

In [160]:
! pip install bert-tensorflow==1.0.1

In [161]:
#Example on first observation in the training set
print("Sentence : ", train_InputExamples.iloc[0].text_a)
print("-"*30)
print("Tokens : ", tokenizer.tokenize(train_InputExamples.iloc[0].text_a))
print("-"*30)
print("Input IDs : ", train_features[0].input_ids)
print("-"*30)
print("Input Masks : ", train_features[0].input_mask)
print("-"*30)
print("Segment IDs : ", train_features[0].segment_ids)

Sentence :  VVeritiv feritrv Operating Company elnvoice FSC SGSNA COC 003630 SFI SGS SFI COC 0037 PEFC SGS PEFC COC 0208 VERITIV OPERATING COMPANY 6120 South Gilmore Road Fairfield OH 45014 VALASSIS PO BOX 159 VENDOR FINANCIAL SERVICES WINDSOR CT 06095 0159 Invoice No Invoice Date Account No 579 75477630 05 02 2019 313050 Due Date Total Amount Due Enter Amount Paid 06 01 2019 338 86 VERITIV OPERATING COMPANY 7472 COLLECTION CENTER DRIVE CHICAGO IL 60693 Detach and return this portion with remittance Retain bottom portion for your records S H I P T 0 VALASSIS INSERTS 35955 SCHOOLCRAFT LIVONIA MI 48150 1215 S 0 L D T 0 Invoice No Invoice Date Customer P O Order Date VALASSIS PO BOX 159 VENDOR FINANCIAL SERVICES WINDSOR CT 06095 0159 Page 1 1 Type of Sale WAREHOUSE Terms 579 75477630 05 02 2019 1062167 05 01 2019 NET 30 DAYS Product No Description References Weight Qty Ordered Qty Shipped Unit Meas Unit Price Unit Extended Meas Price GP SALES ORDER NO 75477630 10728024 Discount Allowed On

# BERT: Creating A Multi-Class Classifier Model

In [162]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  
  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]
  # with tf.Session() as sess:
  output_layer1 = bert_outputs["pooled_output"]
  # output_layer1 = 999
  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.8)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs, output_layer1)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

In [163]:
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        
        return {
            "eval_accuracy": accuracy,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg,
            }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs, output_layer) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)
      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels,
          'pooled_output': output_layer
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

In [164]:
BATCH_SIZE = 16
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 1.0
# Warmup is a period of time where the learning rate is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 300
SAVE_SUMMARY_STEPS = 100

# Compute train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Specify output directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

# Specify output directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [165]:
num_train_steps, len(label_list)

(288, 4)

In [166]:
#Initializing the model and the estimator
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


In [167]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

# Create an input function for validating. drop_remainder = True for using TPUs.
val_input_fn = run_classifier.input_fn_builder(
    features=val_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

# BERT: Fine Tuning Training & Evaluating

In [168]:
#Training the model
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!


/home/komleva/anaconda3/envs/torchtext/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Training took time  0:25:52.535158


The accuracy for the fine tuned BERT model

In [169]:
#Evaluating the model with Validation set
estimator.evaluate(input_fn=val_input_fn, steps=None)

/home/komleva/anaconda3/envs/torchtext/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


{'eval_accuracy': 0.7998267,
 'false_negatives': 73.0,
 'false_positives': 104.0,
 'loss': 0.55226713,
 'true_negatives': 50.0,
 'true_positives': 927.0,
 'global_step': 288}

In [170]:
#! pip install https://storage.googleapis.com/tensorflow/mac/cpu/tensorflow-1.8.0-py3-none-any.whl


# BERT: Get The Vector Transformations from the Fine Tuned BERT

In [171]:
# A method to get predictions
def getPrediction(in_sentences, type_output = "features"):
  #A list to map the actual labels to the predictions
  labels = np.unique(train['label'])
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] 
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  #Predicting the classes 
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  if type_output == "features":
    return [prediction['pooled_output'] for _,prediction in enumerate(predictions) ]
  else:
    return ([(sentence, prediction['probabilities'],
              prediction['labels'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)])


In [172]:
tf.compat.v1.logging.set_verbosity(tf.logging.ERROR)
MAX_SEQ_LENGTH

200

In [173]:
train_df.shape, val_df.shape

((4623, 2), (1154, 2))

Now extracting the representations:

In [174]:
%%time
tr_emb = np.apply_along_axis(getPrediction, 0,np.array(train_df[DATA_COLUMN]))


CPU times: user 7h 26min 51s, sys: 38min 6s, total: 8h 4min 58s
Wall time: 6min 8s


In [175]:
%%time
val_emb = np.apply_along_axis(getPrediction, 0,np.array(val_df[DATA_COLUMN]))
val_emb.shape

CPU times: user 1h 51min 54s, sys: 9min 12s, total: 2h 1min 6s
Wall time: 1min 34s


(1154, 768)

In [176]:
val_emb.shape, tr_emb.shape

((1154, 768), (4623, 768))

and make the dataset for train and val:

In [177]:
aux = -1
len_l = 0
train_x = {}
for l, emb in zip(index_l, tr_emb):
  if l in train_x.keys():
    train_x[l]  =np.vstack([train_x[l], emb])
  else:
    train_x[l] = [emb]

len(train_x.keys())


3760

In [178]:
train_l_final = []
label_l_final = []
for k in train_x.keys():
  train_l_final.append(train_x[k])
  label_l_final.append(train.loc[k]['label'])

df_train = pd.DataFrame({'emb': train_l_final, 'label': label_l_final, })
df_train.head()

,emb,label
0,"[[0.50509787, -0.122917235, 0.88433576, 0.0955...",3
1,"[[0.5403556, 0.07508825, 0.80675274, 0.4831976...",3
2,"[[0.5257652, -0.10080789, 0.83823156, 0.431485...",3
3,"[[0.45012447, 0.04453288, 0.90325177, 0.316509...",3
4,"[[0.38010636, -0.11642724, 0.96094316, 0.15046...",3


In [179]:
aux = -1
len_l = 0
val_x = {}

for l, emb in zip(val_index_l, val_emb):
  if l in val_x.keys():
    val_x[l]  =np.vstack([val_x[l], emb])
  else:
    val_x[l] = [emb]


val_l_final = []
vlabel_l_final = []
for k in val_x.keys():
  val_l_final.append(val_x[k])
  vlabel_l_final.append(val.loc[k]['label'])

df_val = pd.DataFrame({'emb': val_l_final, 'label': vlabel_l_final})
df_val.head()

,emb,label
0,"[[0.52215177, 0.21213302, 0.9069279, 0.3265816...",3
1,"[[0.41261834, 0.012177956, 0.9348166, 0.254106...",3
2,"[[0.44188797, -0.060664732, 0.9067899, 0.22350...",3
3,"[[0.5136896, 0.084379256, 0.9371997, 0.2767630...",3
4,"[[0.84879386, -0.30391622, -0.08012636, -0.439...",2


In [180]:
df_val, df_test = train_test_split(df_val, test_size=0.4, random_state=35)

# LSTM: Creating the Final Model

In [181]:
from keras import layers
text_input = Input(shape=(None,768,), dtype='float32', name='text')

l_mask = layers.Masking(mask_value=-99.)(text_input)
# Which we encoded in a single vector via a LSTM
encoded_text = layers.LSTM(100,)(l_mask)
out_dense = layers.Dense(30, activation='relu')(encoded_text)
# And we add a softmax classifier on top
out = layers.Dense(len(label_list), activation='softmax')(out_dense)
# At model instantiation, we specify the input and the output:
model = Model(text_input, out)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text (InputLayer)            (None, None, 768)         0         
_________________________________________________________________
masking_3 (Masking)          (None, None, 768)         0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               347600    
_________________________________________________________________
dense_5 (Dense)              (None, 30)                3030      
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 124       
Total params: 350,754
Trainable params: 350,754
Non-trainable params: 0
_________________________________________________________________


In [182]:
df_train.shape, df_val.shape, df_test.shape

((3760, 2), (564, 2), (377, 2))

The generator functions:

In [183]:
def train_generator(df):
    x_list= df['emb'].to_list()
    y_list =  df.label.to_list()
    # Generate batches
    while True:
        for b in range(batches_per_epoch):
            longest_index = (b + 1) * batch_size - 1
            timesteps = len(max(df['emb'].to_list()[:(b + 1) * batch_size][-batch_size:], key=len))
            x_train = np.full((batch_size, timesteps, num_features), -99.)
            y_train = np.zeros((batch_size,  1))
            
            for i in range(batch_size):
                li = b * batch_size + i
                x_train[i, 0:len(x_list[li]), :] = x_list[li]
                y_train[i] = y_list[li]
            yield x_train, y_train

In [191]:
from tensorflow.python.compiler.tensorrt import trt_convert as trt



num_sequences = len(df_train['emb'].to_list())
print(num_sequences)
batch_size = 5
batches_per_epoch =  752

assert batch_size * batches_per_epoch == num_sequences
num_features= 768


3760


In [194]:
num_sequences_val = len(df_val['emb'].to_list())
print(num_sequences_val)
batch_size_val = 12
batches_per_epoch_val = 47
assert batch_size_val * batches_per_epoch_val == num_sequences_val
num_features= 768
def val_generator(df):
    x_list= df['emb'].to_list()
    y_list =  df.label.to_list()
    # Generate batches
    while True:
        for b in range(batches_per_epoch_val):
            longest_index = (b + 1) * batch_size_val - 1
            timesteps = len(max(df['emb'].to_list()[:(b + 1) * batch_size_val][-31:], key=len))
            # print(len(df_train['emb'].to_list()[:b+batch_size][-7:]))
            x_train = np.full((batch_size_val, timesteps, num_features), -99.)
            y_train = np.zeros((batch_size_val,  1))
            for i in range(batch_size_val):
                li = b * batch_size_val + i
                # print("li", li)
                # print(x_train[i, 0:len(x_list[li]), :].shape, len(x_list[li]))
                x_train[i, 0:len(x_list[li]), :] = x_list[li]
                y_train[i] = y_list[li]
            yield x_train, y_train

564


# LSTM Final Model: Training

In [196]:
from keras.callbacks import ReduceLROnPlateau
call_reduce = ReduceLROnPlateau(monitor='val_acc', factor=0.95, patience=3, verbose=2,
                                mode='auto', min_delta=0.01, cooldown=0, min_lr=0)

In [197]:
model.fit_generator(train_generator(df_train), steps_per_epoch=batches_per_epoch, epochs=10,
                    validation_data=val_generator(df_val), validation_steps=batches_per_epoch_val, callbacks =[call_reduce] )

Epoch 1/10
752/752 [==============================] - 5s 7ms/step - loss: 0.3929 - acc: 0.8649 - val_loss: 0.0822 - val_acc: 0.8227
Epoch 2/10
752/752 [==============================] - 5s 7ms/step - loss: 0.3882 - acc: 0.8668 - val_loss: 0.0813 - val_acc: 0.8262
Epoch 3/10
752/752 [==============================] - 5s 7ms/step - loss: 0.3862 - acc: 0.8681 - val_loss: 0.0759 - val_acc: 0.8262
Epoch 4/10
752/752 [==============================] - 5s 7ms/step - loss: 0.3850 - acc: 0.8697 - val_loss: 0.0765 - val_acc: 0.8262

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0009500000451225787.
Epoch 5/10
752/752 [==============================] - 5s 7ms/step - loss: 0.3794 - acc: 0.8718 - val_loss: 0.0845 - val_acc: 0.8262
Epoch 6/10
752/752 [==============================] - 5s 7ms/step - loss: 0.3757 - acc: 0.8739 - val_loss: 0.0884 - val_acc: 0.8262
Epoch 7/10
752/752 [==============================] - 5s 7ms/step - loss: 0.3737 - acc: 0.8761 - val_loss: 0.0882 - val_acc: 0.

# LSTM Final Model: Evaluation



In [ ]:
num_sequences_val = len(df_test['emb'].to_list())
batch_size_val = 4
batches_per_epoch_val = 343
assert batch_size_val * batches_per_epoch_val == num_sequences_val
num_features= 768
model.evaluate_generator(val_generator(df_test), steps= batches_per_epoch_val)

[0.41612950315069047, 0.8731778425655977]